In [1]:
import os

os.chdir("/home/hack12/ensemble-ai2024/")

In [2]:
import random
from pathlib import Path
from taskdataset import TaskDataset
import hydra
import numpy as np
import torch
import wandb
from omegaconf import omegaconf, OmegaConf
from torch import optim, nn
from torch.utils.data import DataLoader
from torchvision.transforms import RandomAffine
from tqdm import tqdm
import random
from src.config import Config
# from src.data.custom_dataset import EncodingsToLabels, EncodingsDataset
from src.models.linear_head import Net, MapperNet
from src.transforms.affine import AffineTransform, AffineAndPadAndShuffleTransform, PadAndShuffleTransform
from src.transforms.binary import BinaryTransform
import json
from typing import List
import requests
import argparse
from torch.utils.data import Dataset
# import end2end_stealing.vision_transformer as vits
import torchvision.models as models
import torchvision
import numpy as np

/home/hack12/ensemble-ai2024/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def train_binary(epoch, model, train_loader, criterion, optimizer, device):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        output = nn.Sigmoid()(output)
        # pred = (output > 0.5).float()
        target = target.to(device)

        # print('pred', pred)
        # print('target', target)
        loss = criterion(output, target.to(device))

        # print(loss)
        loss.backward()
        optimizer.step()
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
               100. * batch_idx / len(train_loader), loss.item()))

In [4]:
class EncodingsDataset(Dataset):
    def __init__(self, transformed_encodings, encodings):
        self.transformed_encodings = torch.from_numpy(transformed_encodings.astype('float32'))
        self.encodings = torch.from_numpy(encodings.astype('float32'))

    def __len__(self):
        return len(self.transformed_encodings)

    def __getitem__(self, idx):
        return self.transformed_encodings[idx], self.encodings[idx]

In [5]:
class MapperNet(nn.Module):
    def __init__(self, n_inputs, n_outputs, n_hidden):
        super().__init__()
        self.fc1 = nn.Linear(n_inputs, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_outputs)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [6]:
SEED = 0
torch.manual_seed(SEED)
# torch.use_deterministic_algorithms(True)
torch.backends.cudnn.benchmark = False
random.seed(SEED)
np.random.seed(SEED)

device = torch.device(f"cuda") if torch.cuda.is_available() else torch.device("cpu")

In [7]:
dataset = torch.load('/home/hack12/ensemble-ai2024/src/data/SybilAttack.pt')
dataset_ids = dataset.ids

In [8]:
learning_rate = 3e-4
batch_size = 32
mapper_epochs = 150

In [9]:
overlap = 1000

In [10]:
results = {}

In [11]:
dataset_ids = dataset.ids

# np.save('ids_A', ids_A)
ids_A = np.load('ids_A.npy')
# ids_A = np.array(random.sample(dataset_ids, 2000))
# ids_A = get_wodzes(features, dataset_ids, n=2000)
unused_ids = [id for id in dataset_ids if id not in ids_A]

In [12]:
directory_path = '/home/hack12/ensemble-ai2024/src/data/task2_dane/'

In [13]:
files = os.listdir(directory_path)

# Filter files that start with 'B'
filtered_files = [file for file in files if file.startswith('B')]


In [14]:
A_data = np.load('/home/hack12/ensemble-ai2024/src/data/task2_dane/A.npy')

In [15]:
B_data_queries = [np.load(directory_path+file_name) for file_name in filtered_files]

In [16]:
B_data_train = [B_data[:overlap] for B_data in B_data_queries]

In [17]:
B_data_test = [B_data[overlap:] for B_data in B_data_queries]

In [18]:
A_data_train = A_data[:overlap]

In [19]:
results['ids'] = dataset_ids
results['features'] = A_data

In [20]:
MODEL_OUT_IN_SIZE = A_data_train.shape[1]
HIDDEN = 2*384

In [21]:
for B_data_train, B_data_test in zip(B_data_train, B_data_test):

    train_dataset = EncodingsDataset(B_data_train, A_data_train)

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    mapper = MapperNet(MODEL_OUT_IN_SIZE,
                        MODEL_OUT_IN_SIZE,
                        n_hidden=HIDDEN).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(mapper.parameters(), lr=learning_rate)

    for epoch in range(1, mapper_epochs):
        train_binary(epoch, mapper, train_loader, criterion, optimizer, device=device)

    reconstructed_encodings_test = []
    with torch.no_grad():
        for encoding in tqdm(B_data_test):
            encoding = torch.Tensor(encoding)
            reconstructed_encoding = mapper(encoding.to(device))
            reconstructed_encoding = nn.Sigmoid()(reconstructed_encoding)
            reconstructed_encoding_pred = (reconstructed_encoding > 0.5).float()
            reconstructed_encodings_test.append(reconstructed_encoding_pred.detach().to("cpu"))
        reconstructed_encodings_test = torch.cat(reconstructed_encodings_test).reshape(
            len(B_data_test), -1)

    results['features'] = np.concatenate((results['features'], reconstructed_encodings_test.numpy()))

Train Epoch: 1 [248/1000 (97%)]	Loss: 0.143060
Train Epoch: 2 [248/1000 (97%)]	Loss: 0.121494
Train Epoch: 3 [248/1000 (97%)]	Loss: 0.136259
Train Epoch: 4 [248/1000 (97%)]	Loss: 0.122255
Train Epoch: 5 [248/1000 (97%)]	Loss: 0.114954
Train Epoch: 6 [248/1000 (97%)]	Loss: 0.136903
Train Epoch: 7 [248/1000 (97%)]	Loss: 0.123580
Train Epoch: 8 [248/1000 (97%)]	Loss: 0.115457
Train Epoch: 9 [248/1000 (97%)]	Loss: 0.112064
Train Epoch: 10 [248/1000 (97%)]	Loss: 0.134226
Train Epoch: 11 [248/1000 (97%)]	Loss: 0.110070
Train Epoch: 12 [248/1000 (97%)]	Loss: 0.125304
Train Epoch: 13 [248/1000 (97%)]	Loss: 0.107539
Train Epoch: 14 [248/1000 (97%)]	Loss: 0.115804
Train Epoch: 15 [248/1000 (97%)]	Loss: 0.117232
Train Epoch: 16 [248/1000 (97%)]	Loss: 0.102343
Train Epoch: 17 [248/1000 (97%)]	Loss: 0.123610
Train Epoch: 18 [248/1000 (97%)]	Loss: 0.116690
Train Epoch: 19 [248/1000 (97%)]	Loss: 0.125852
Train Epoch: 20 [248/1000 (97%)]	Loss: 0.118083
Train Epoch: 21 [248/1000 (97%)]	Loss: 0.119363
T

100%|██████████| 1000/1000 [00:00<00:00, 4138.99it/s]


Train Epoch: 1 [248/1000 (97%)]	Loss: 0.130339
Train Epoch: 2 [248/1000 (97%)]	Loss: 0.108173
Train Epoch: 3 [248/1000 (97%)]	Loss: 0.111863
Train Epoch: 4 [248/1000 (97%)]	Loss: 0.123611
Train Epoch: 5 [248/1000 (97%)]	Loss: 0.120428
Train Epoch: 6 [248/1000 (97%)]	Loss: 0.134095
Train Epoch: 7 [248/1000 (97%)]	Loss: 0.123275
Train Epoch: 8 [248/1000 (97%)]	Loss: 0.126399
Train Epoch: 9 [248/1000 (97%)]	Loss: 0.125544
Train Epoch: 10 [248/1000 (97%)]	Loss: 0.110102
Train Epoch: 11 [248/1000 (97%)]	Loss: 0.113284
Train Epoch: 12 [248/1000 (97%)]	Loss: 0.133868
Train Epoch: 13 [248/1000 (97%)]	Loss: 0.114049
Train Epoch: 14 [248/1000 (97%)]	Loss: 0.090580
Train Epoch: 15 [248/1000 (97%)]	Loss: 0.109771
Train Epoch: 16 [248/1000 (97%)]	Loss: 0.105179
Train Epoch: 17 [248/1000 (97%)]	Loss: 0.108756
Train Epoch: 18 [248/1000 (97%)]	Loss: 0.120234
Train Epoch: 19 [248/1000 (97%)]	Loss: 0.110096
Train Epoch: 20 [248/1000 (97%)]	Loss: 0.103112
Train Epoch: 21 [248/1000 (97%)]	Loss: 0.099237
T

100%|██████████| 1000/1000 [00:00<00:00, 4162.81it/s]


Train Epoch: 1 [248/1000 (97%)]	Loss: 0.105409
Train Epoch: 2 [248/1000 (97%)]	Loss: 0.113872
Train Epoch: 3 [248/1000 (97%)]	Loss: 0.127211
Train Epoch: 4 [248/1000 (97%)]	Loss: 0.102667
Train Epoch: 5 [248/1000 (97%)]	Loss: 0.127603
Train Epoch: 6 [248/1000 (97%)]	Loss: 0.131773
Train Epoch: 7 [248/1000 (97%)]	Loss: 0.106706
Train Epoch: 8 [248/1000 (97%)]	Loss: 0.126449
Train Epoch: 9 [248/1000 (97%)]	Loss: 0.114081
Train Epoch: 10 [248/1000 (97%)]	Loss: 0.137891
Train Epoch: 11 [248/1000 (97%)]	Loss: 0.127847
Train Epoch: 12 [248/1000 (97%)]	Loss: 0.135523
Train Epoch: 13 [248/1000 (97%)]	Loss: 0.121420
Train Epoch: 14 [248/1000 (97%)]	Loss: 0.120673
Train Epoch: 15 [248/1000 (97%)]	Loss: 0.118473
Train Epoch: 16 [248/1000 (97%)]	Loss: 0.105598
Train Epoch: 17 [248/1000 (97%)]	Loss: 0.146137
Train Epoch: 18 [248/1000 (97%)]	Loss: 0.117669
Train Epoch: 19 [248/1000 (97%)]	Loss: 0.107033
Train Epoch: 20 [248/1000 (97%)]	Loss: 0.091500
Train Epoch: 21 [248/1000 (97%)]	Loss: 0.107318
T

...

100%|██████████| 1000/1000 [00:00<00:00, 4199.87it/s]


Train Epoch: 1 [248/1000 (97%)]	Loss: 0.121202
Train Epoch: 2 [248/1000 (97%)]	Loss: 0.128147
Train Epoch: 3 [248/1000 (97%)]	Loss: 0.104579
Train Epoch: 4 [248/1000 (97%)]	Loss: 0.125520
Train Epoch: 5 [248/1000 (97%)]	Loss: 0.118629
Train Epoch: 6 [248/1000 (97%)]	Loss: 0.119723
Train Epoch: 7 [248/1000 (97%)]	Loss: 0.141400
Train Epoch: 8 [248/1000 (97%)]	Loss: 0.101885
Train Epoch: 9 [248/1000 (97%)]	Loss: 0.100371
Train Epoch: 10 [248/1000 (97%)]	Loss: 0.130887
Train Epoch: 11 [248/1000 (97%)]	Loss: 0.124935
Train Epoch: 12 [248/1000 (97%)]	Loss: 0.119084
Train Epoch: 13 [248/1000 (97%)]	Loss: 0.111048
Train Epoch: 14 [248/1000 (97%)]	Loss: 0.103915
Train Epoch: 15 [248/1000 (97%)]	Loss: 0.103323
Train Epoch: 16 [248/1000 (97%)]	Loss: 0.099528
Train Epoch: 17 [248/1000 (97%)]	Loss: 0.134078
Train Epoch: 18 [248/1000 (97%)]	Loss: 0.103097
Train Epoch: 19 [248/1000 (97%)]	Loss: 0.109523
Train Epoch: 20 [248/1000 (97%)]	Loss: 0.130832
Train Epoch: 21 [248/1000 (97%)]	Loss: 0.111784
T

100%|██████████| 1000/1000 [00:00<00:00, 4153.85it/s]


Train Epoch: 1 [248/1000 (97%)]	Loss: 0.117562
Train Epoch: 2 [248/1000 (97%)]	Loss: 0.121716
Train Epoch: 3 [248/1000 (97%)]	Loss: 0.111761
Train Epoch: 4 [248/1000 (97%)]	Loss: 0.129669
Train Epoch: 5 [248/1000 (97%)]	Loss: 0.141380
Train Epoch: 6 [248/1000 (97%)]	Loss: 0.114193
Train Epoch: 7 [248/1000 (97%)]	Loss: 0.132563
Train Epoch: 8 [248/1000 (97%)]	Loss: 0.139135
Train Epoch: 9 [248/1000 (97%)]	Loss: 0.110132
Train Epoch: 10 [248/1000 (97%)]	Loss: 0.107963
Train Epoch: 11 [248/1000 (97%)]	Loss: 0.132092
Train Epoch: 12 [248/1000 (97%)]	Loss: 0.121039
Train Epoch: 13 [248/1000 (97%)]	Loss: 0.133273
Train Epoch: 14 [248/1000 (97%)]	Loss: 0.126655
Train Epoch: 15 [248/1000 (97%)]	Loss: 0.119501
Train Epoch: 16 [248/1000 (97%)]	Loss: 0.127003
Train Epoch: 17 [248/1000 (97%)]	Loss: 0.111740
Train Epoch: 18 [248/1000 (97%)]	Loss: 0.117736
Train Epoch: 19 [248/1000 (97%)]	Loss: 0.104720
Train Epoch: 20 [248/1000 (97%)]	Loss: 0.110536
Train Epoch: 21 [248/1000 (97%)]	Loss: 0.122760
T

100%|██████████| 1000/1000 [00:00<00:00, 4206.35it/s]


In [22]:
len(results['features'])

20000

In [23]:
np.savez(
        "example_submission_binarowiak.npz",
        ids=results['ids'],
        representations=results['features'],
    )

In [24]:
def sybil_submit(binary_or_affine: str, path_to_npz_file: str):

    TEAM_TOKEN = "8J40ASDQOjfeeSKL"
    SERVER_URL = "http://34.71.138.79:9090"
    
    if binary_or_affine not in ["binary", "affine"]:
        raise Exception("Invalid endpoint")

    endpoint = f"/sybil/{binary_or_affine}/submit"
    url = SERVER_URL + endpoint

    with open(path_to_npz_file, "rb") as f:
        response = requests.post(url, files={"file": f}, headers={"token": TEAM_TOKEN})

    if response.status_code == 200:
        print("OK")
        print(response.json())
    else:
        print(
            f"Request submit failed. Status code: {response.status_code}, content: {response.json()}"
        )

In [25]:
sybil_submit('binary', '/home/hack12/ensemble-ai2024/example_submission_binarowiak.npz')

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))